# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)
data.groupby(['race', 'call']).id.size()

race  call
b     0.0     2278
      1.0      157
w     0.0     2200
      1.0      235
Name: id, dtype: int64

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>
    1. What test is appropriate for this problem? Does CLT apply? <br>
    A two-tail z test is appropriate here to compare the callback rate of different races/names.
    CLT applies b/c the sample size is large enough.
    <br>
    <br>
    2. What are the null and alternate hypotheses?<br>
    H<sub>0</sub>: p<sub>b</sub> = p<sub>w</sub><br>
    H<sub>A</sub>: p<sub>b</sub> $\neq$ p<sub>w</sub><br>
    Where p<sub>b</sub> is the proportion of blacks get callback, and p<sub>w</sub> is the proportion of whites get callback
 
</p>
</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

In [18]:
# 3. Compute margin of error, confidence interval, and p-value. 
#    Try using both the bootstrapping and the frequentist statistical approaches.

# Bootstrap
w_call_rate = len(w[w.call == 1]) / len(w)
b_call_rate = len(b[b.call == 1]) / len(b)
diff_call_rate = w_call_rate - b_call_rate

bs_diff_call_rate = np.empty(10000)
for i in range(10000):
    bs_call = np.random.choice(data.call, size=len(data.call))
    bs_w_call = bs_call[:len(w)]
    bs_b_call = bs_call[len(w):]
    bs_w_call_rate = np.sum(bs_w_call == 1) / len(bs_w_call)
    bs_b_call_rate = np.sum(bs_b_call == 1) / len(bs_b_call)
    bs_diff_call_rate[i] = bs_w_call_rate - bs_b_call_rate

p_value_bs = np.sum(bs_diff_call_rate >= diff_call_rate) / len(bs_diff_call_rate)
#print('95% confidence interval of difference between call back rate of whites and of blacks is: '\
#     + str(np.percentile(bs_diff_call_rate, [2.5, 97.5])))
print('Observed difference is: ' + str(diff_call_rate))
print('p value is: ' + str(p_value_bs))
print('Reject null hypothesis. Conclusion: there is difference between the probability of getting call back by race.')

Observed difference is: 0.032032854209445585
p value is: 0.0
Reject null hypothesis. Conclusion: there is difference between the probability of getting call back by race.


In [19]:
# frequentist statistical approaches
all_call_rate = len(data[data.call == 1]) / len(data.call)
z_score = diff_call_rate / np.sqrt(all_call_rate*(1-all_call_rate)*((1/len(w))+(1/len(b))))
print(z_score)

4.108412152434346


p_value << 0.05, reject null hypothesis. Conclusion: there is difference between the probability of getting call back by race.

In [21]:
moe = 1.96 * np.sqrt(all_call_rate*(1-all_call_rate)*((1/len(w))+(1/len(b))))
print('95% confidence interval of difference between call back rate of whites and of blacks is: [' \
     + str(diff_call_rate - moe) + ', ' + str(diff_call_rate + moe) + ']')

95% confidence interval of difference between call back rate of whites and of blacks is: [0.01675094189855149, 0.04731476652033968]


<div class="span5 alert alert-success">
<p> 
    4. Write a story describing the statistical significance in the context or the original problem.<br>
    <br>The above hypothesis testing results show that there are statistically significant differnce between the probability of white people getting callback after submitting resume and the probability of black people getting callback after submitting resume. <br>
    The p-values of both tests are less than 0.05, which means under the null hypothesis that people of different races have the same probability of getting callback, the probability of getting the observed difference (~0.03) is less than 5%, so we have enough evidence to reject the null hypothesis. Therefore, the conclusion is that race does have a significant impact on the rate of callbacks.<br>
    <br>
    5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?<br>
    <br>The analysis shows that race/name factor has impact on callback rate, but it doesn't mean that race/name is the only factor or the most important factor in callback success. The data involved in the tests contains only race and callback info, therefore the analysis is not complete. I will continue to investigate the relationship between callback rate and other factors such as education, years of experience etc.
    
</p>
</div>